In [1]:
# Import some python libraries
%matplotlib inline

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import intake

/Users/aaron.spring/anaconda3/envs/xr/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
# Setup a dask cluster
from dask.distributed import Client
from dask_kubernetes import KubeCluster

cluster = KubeCluster(n_workers=10)
cluster

In [ ]:
client = Client(cluster)
client

In [ ]:
col = intake.open_esm_datastore("https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json")

In [ ]:
variable_id = 'tas'
table_id='Amon'
source_id = 'CESM1-1-CAM5-CMIP5'

# Get hindcast experiment

In [ ]:
cat_cmip = col.search(experiment_id=['dcppA-hindcast'],
                 table_id=table_id,       
                 variable_id=variable_id,
                 source_id=source_id,
                     )
cat_cmip.df['dcpp_init_year'] = cat_cmip.df.dcpp_init_year.astype(int)
cat_cmip

In [ ]:
members = sorted(list(cat_cmip.df.member_id.unique()))[:5]
inits = list(np.arange(1970,1990))

In [ ]:
cat_cmip = col.search(experiment_id=['dcppA-hindcast'],
                 table_id=table_id,       
                 variable_id=variable_id,
                 source_id=source_id,
                 member_id = members,
                 dcpp_init_year=inits
                     )
cat_cmip.df['dcpp_init_year'] = cat_cmip.df.dcpp_init_year.astype(int)

In [ ]:
def pre(ds,var='tas'):
    ds['time']=np.arange(1,1+ds.time.size)
    return ds[var].squeeze().to_dataset(name=var)

In [ ]:
dset_dict = cat_cmip.to_dataset_dict(zarr_kwargs={'consolidated': True},preprocess=pre,cdf_kwargs={'decode_times':False})
list(dset_dict.keys())

In [ ]:
hind = dset_dict['DCPP.NCAR.CESM1-1-CAM5-CMIP5.dcppA-hindcast.Amon.gn']

In [ ]:
hind = hind.rename({'member_id':'member','dcpp_init_year':'init','time':'lead'})

In [ ]:
# throw away first two leads and create annual means
hind=hind.isel(lead=slice(2,None))
hind['lead']=xr.cftime_range(start='2000',freq='MS',periods=hind.lead.size)
hind_ym = hind.groupby('lead.year').mean().rename({'year':'lead'})
hind_ym['lead']=np.arange(1,1+hind_ym.lead.size)

In [ ]:
hind_ym = hind_ym.chunk({'init':-1,'member':-1,'lead':1}).persist()

hind_ym = hind_ym['tas']

## Get historical

In [ ]:
cat_cmip_hist = col.search(experiment_id=['historical','ssp45'],  # pick the `historical` forcing experiment
                 table_id=table_id,       
                 variable_id=variable_id,
                 source_id='CESM2', # not exactly CESM-LE
                 )

In [ ]:
dset_dict = cat_cmip_hist.to_dataset_dict(zarr_kwargs={'consolidated': True})
dset_dict.keys()

In [ ]:
hist = dset_dict['CMIP.NCAR.CESM2.historical.Amon.gn']

hist = hist[variable_id].squeeze()

hist = hist.rename({'member_id':'member'})

hist = hist.sel(time=slice('1960','2015'))

hist_ym = hist.groupby('time.year').mean().rename({'year':'time'})

hist_ym.data

## Get assimilation

In [ ]:
cat_cmip_hist = col.search(experiment_id=['dcppA-assim'],  # pick the `historical` forcing experiment
                 table_id=table_id,       
                 variable_id=variable_id,
                 source_id=source_id
                          )

In [ ]:
cat_cmip_hist.df.source_id.unique()

## Get observations

In [3]:
!wget https://crudata.uea.ac.uk/cru/data/temperature/HadCRUT.4.6.0.0.median.nc

--2020-02-18 21:22:07--  https://crudata.uea.ac.uk/cru/data/temperature/HadCRUT.4.6.0.0.median.nc
Aufl"osen des Hostnamens crudata.uea.ac.uk (crudata.uea.ac.uk)... 139.222.133.100
Verbindungsaufbau zu crudata.uea.ac.uk (crudata.uea.ac.uk)|139.222.133.100|:443 ... verbunden.
HTTP-Anforderung gesendet, auf Antwort wird gewartet ... 200 OK
L"ange: 21468796 (20M) [application/x-netcdf]
Wird in >>HadCRUT.4.6.0.0.median.nc<< gespeichert.

dCRUT.4.6.0.0.media   6%[>                   ]   1.33M   339KB/s    ETA 66s    ^C


In [ ]:
obs = xr.open_dataset('HadCRUT.4.6.0.0.median.nc')

In [ ]:
obs = obs.rename({'latitude':'lat','longitude':'lon','temperature_anomaly':variable_id})[variable_id]

obs = obs.sel(time=slice('1960',None))

obs_ym = obs.groupby('time.year').mean().rename({'year':'time'})

# regrid

In [ ]:
import xesmf as xe

def regrid(ds):
    ds_out = xe.util.grid_global(5,5)
    regridder = xe.Regridder(ds, ds_out, 'bilinear')
    ds_out = regridder(ds)
    return ds_out

In [ ]:
hind_ym_regridded = regrid(hind_ym)
hist_ym_regridded = regrid(hist_ym)
obs_ym_regridded = regrid(obs_ym)

### check the inputs

In [ ]:
hind_ym_regridded.isel(member=0,lead=2,init=2).plot()

In [ ]:
hist_ym_regridded.isel(member=0,time=2).plot()

In [ ]:
obs_ym_regridded.isel(time=2).plot()

## Skill

In [ ]:
from climpred.prediction import compute_hindcast

## Significant skill

In [ ]:
from climpred.bootstrap import bootstrap_hindcast